In [3]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

# Define paths
data_dir = '/Users/samuelsetsofia/dev/projects/EyeDisease_Classifier_DeepLearning/dataset'
classes = ['cataract', 'glaucoma', 'diabetic_retinopathy', 'normal']

# Create train, val, and test folders
output_dir = '/Users/samuelsetsofia/dev/projects/EyeDisease_Classifier_DeepLearning/output_dir'
os.makedirs(output_dir, exist_ok=True)
for folder in ['train', 'val', 'test']:
    for cls in classes:
        os.makedirs(os.path.join(output_dir, folder, cls), exist_ok=True)

# Split files
for cls in classes:
    class_path = os.path.join(data_dir, cls)
    files = np.array(os.listdir(class_path))
    train_files, temp_files = train_test_split(files, test_size=0.3, random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)

    for file_list, folder in zip([train_files, val_files, test_files], ['train', 'val', 'test']):
        for file in file_list:
            shutil.copy(os.path.join(class_path, file), os.path.join(output_dir, folder, cls))

### Load and Preprocess Images

In [4]:
import tensorflow as tf

train_dir = '/Users/samuelsetsofia/dev/projects/EyeDisease_Classifier_DeepLearning/output_dir/train'
val_dir = '/Users/samuelsetsofia/dev/projects/EyeDisease_Classifier_DeepLearning/output_dir/val'
test_dir = '/Users/samuelsetsofia/dev/projects/EyeDisease_Classifier_DeepLearning/output_dir/test'

batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 2949 files belonging to 4 classes.
Found 633 files belonging to 4 classes.
Found 635 files belonging to 4 classes.


### Model 1: VGG16 (Transfer Learning)
A deep convolutional neural network architecture that is effective for image classification tasks.

In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

base_model = VGG16(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model_vgg16 = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # 4 classes
])

model_vgg16.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_vgg16 = model_vgg16.fit(train_ds, validation_data=val_ds, epochs=10)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step
Epoch 1/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 244s 3s/step - accuracy: 0.6266 - loss: 10.9791 - val_accuracy: 0.7788 - val_loss: 0.5804
Epoch 2/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 245s 3s/step - accuracy: 0.7895 - loss: 0.5918 - val_accuracy: 0.8436 - val_loss: 0.4252
Epoch 3/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 235s 3s/step - accuracy: 0.8059 - loss: 0.4783 - val_accuracy: 0.8468 - val_loss: 0.4794
Epoch 4/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 238s 3s/step - accuracy: 0.8082 - loss: 0.4721 - val_accuracy: 0.8531 - val_loss: 0.3756
Epoch 5/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 242s 3s/step - accuracy: 0.8297 - loss: 0.4243 - val_accuracy: 0.8799 - val_loss: 0.3650
Epoch 6/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 234s 3s/step - accuracy: 0.8558 - loss: 0.3562 - val_accuracy: 0.8752 - val_loss: 0.3620
Epoch 7/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 233s 3s/step - accuracy: 0.8915 - loss: 0.2973 - val_accuracy: 0.8894 - val_loss: 0.3735
Epoch 8/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 232s 2s/step - accur

### Model 2: ResNet50 (Transfer Learning)
A residual network that enables deeper networks by addressing vanishing gradient issues.

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model_resnet50 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model_resnet50.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_resnet50 = model_resnet50.fit(train_ds, validation_data=val_ds, epochs=10)

### Model 3: MobileNetV2 (Transfer Learning)
A lightweight convolutional neural network suitable for mobile and embedded applications.

In [ ]:
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model_mobilenetv2 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model_mobilenetv2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_mobilenetv2 = model_mobilenetv2.fit(train_ds, validation_data=val_ds, epochs=10)

### Evaluate Model Performance
Confusion Matrix: For detailed class-level performance.
Precision, Recall, F1-Score: For assessing balance between sensitivity and specificity.

In [ ]:
vgg16_loss, vgg16_accuracy = model_vgg16.evaluate(test_ds)
resnet50_loss, resnet50_accuracy = model_resnet50.evaluate(test_ds)
mobilenetv2_loss, mobilenetv2_accuracy = model_mobilenetv2.evaluate(test_ds)

from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

true_labels = np.concatenate([y for x, y in test_ds], axis=0)

vgg16_predictions = np.argmax(model_vgg16.predict(test_ds), axis=1)
resnet50_predictions = np.argmax(model_resnet50.predict(test_ds), axis=1)
mobilenetv2_predictions = np.argmax(model_mobilenetv2.predict(test_ds), axis=1)

# Print classification reports
print("VGG16:")
print(classification_report(true_labels, vgg16_predictions))
print("ResNet50:")
print(classification_report(true_labels, resnet50_predictions))
print("MobileNetV2:")
print(classification_report(true_labels, mobilenetv2_predictions))